In [ ]:
# pip install langchain==0.1.20 langchain-community==0.0.38
# 

In [1]:
from langchain_community.document_loaders import WebBaseLoader

# Static, non-JS website (Wikipedia)
url = "https://en.wikipedia.org/wiki/Retrieval-augmented_generation"

loader = WebBaseLoader(
    web_paths=[url],
)

loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

documents = loader.load()

print(f"Loaded {len(documents)} document(s)")
print()
print("Documents are: ", documents)
print()
print(documents[0].page_content[:500])


Loaded 1 document(s)

Documents are:  [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Retrieval-augmented_generation', 'title': 'Retrieval-augmented generation - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nRetrieval-augmented generation - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " "]
)

chunks = text_splitter.split_documents(documents)
print(len(chunks))
print(chunks[22].page_content[:300])



43
Evaluation and benchmarks[edit]
RAG systems are commonly evaluated using benchmarks designed to test retrievability, retrieval accuracy and generative quality. Popular datasets include BEIR, a suite of information retrieval tasks across diverse domains, and Natural Questions or Google QA for open-do


In [4]:
# %pip install faiss-cpu

In [5]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="gemma:2b")

embeddings

C:\Users\user\AppData\Local\Temp\ipykernel_31152\1498907739.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


OllamaEmbeddings(base_url='http://localhost:11434', model='gemma:2b', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

vectorstore

In [ ]:
vectorstore.index_to_docstore_id

In [20]:
doc = vectorstore.get_by_ids(['11323c3f-1ec8-4625-95c1-ad8abb60ff1d'])
print(doc)

[Document(id='11323c3f-1ec8-4625-95c1-ad8abb60ff1d', metadata={'source': 'https://en.wikipedia.org/wiki/Retrieval-augmented_generation', 'title': 'Retrieval-augmented generation - Wikipedia', 'language': 'en'}, page_content='References[edit]\n\n^ a b c d e f "What is retrieval-augmented generation?". IBM. 22 August 2023. Retrieved 7 March 2025.\n\n^ a b c d e f "Why Google\'s AI Overviews gets things wrong". MIT Technology Review. 31 May 2024. Retrieved 7 March 2025.\n\n^ a b c d Lewis, Patrick; Perez, Ethan; Piktus, Aleksandra; Petroni, Fabio; Karpukhin, Vladimir; Goyal, Naman; Küttler, Heinrich; Lewis, Mike; Yih, Wen-tau; Rocktäschel, Tim; Riedel, Sebastian; Kiela, Douwe (6 December 2020). Retrieval-augmented generation for knowledge-intensive NLP tasks. International Conference on Neural Information Processing Systems. Red Hook, NY, USA: Curran Associates Inc. ISBN\xa0978-1-7138-2954-6. Retrieved 9 December 2025.')]


In [24]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # remove score_threshold
)

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002689FC94EC0>, search_kwargs={'k': 4})

In [25]:
retriever.invoke("tell me about RAG")

[Document(id='fb81a74c-e0f6-4778-937f-1c3085101ff5', metadata={'source': 'https://en.wikipedia.org/wiki/Retrieval-augmented_generation', 'title': 'Retrieval-augmented generation - Wikipedia', 'language': 'en'}, page_content='vteGenerative AIConcepts\nAutoencoder\nDeep learning\nFine-tuning\nFoundation model\nGenerative adversarial network\nGenerative pre-trained transformer\nLarge language model\nModel Context Protocol\nNeural network\nPrompt engineering\nReinforcement learning from human feedback\nRetrieval-augmented generation\nSelf-supervised learning\nStochastic parrot\nSynthetic data\nTop-p sampling\nTransformer\nVariational autoencoder\nVibe coding\nVision transformer\nWord embedding\nChatbots\nCharacter.ai\nChatGPT\nCopilot\nDeepSeek\nErnie\nGemini\nGrok\nModelsText\nClaude\nGemini\nGemma\nGPT\n1\n2\n3\nJ\n4\n4o\n4.5\n4.1\nOSS\n5\n5.1\n5.2\nLlama\no1\no3\no4-mini\nQwen\nVelvet\nCoding\nClaude Code\nCursor\nDevstral\nGitHub Copilot\nKimi\nQwen3-Coder\nReplit\nImage\nAurora\nFiref

In [26]:
query = "tell me a short intro of RAG and langchain"

retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='8ccea756-1dce-4e41-bdf6-1ed8b8bc382f', metadata={'source': 'https://en.wikipedia.org/wiki/Retrieval-augmented_generation', 'title': 'Retrieval-augmented generation - Wikipedia', 'language': 'en'}, page_content='Retrieval-augmented generation - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n

In [27]:
from langchain_ollama import ChatOllama

llm_model = ChatOllama(model="gemma:2b")

llm_model

ChatOllama(model='gemma:2b')

In [28]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    template="""
        you are an knowledgeable assistant!
        answer to the asked question based on the given context.
        if not sure, clearly say i don't know!

        {context},
        question: {query}
    """,
    input_variables=['context', 'query']
)

template

PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="\n        you are an knowledgeable assistant!\n        answer to the asked question based on the given context.\n        if not sure, clearly say i don't know!\n\n        {context},\n        question: {query}\n    ")

In [29]:
query = "is rag the part of ai / ml?"

retrieved_docs = retriever.invoke(query)

In [30]:
retrieved_docs

[Document(id='8f85b111-aca5-43f7-b8de-f2d787185dc1', metadata={'source': 'https://en.wikipedia.org/wiki/Retrieval-augmented_generation', 'title': 'Retrieval-augmented generation - Wikipedia', 'language': 'en'}, page_content='RAG systems may retrieve factually correct but misleading sources, leading to errors in interpretation. In some cases, an LLM may extract statements from a source without considering its context, resulting in an incorrect conclusion. Additionally, when faced with conflicting information RAG models may struggle to determine which source is accurate. The worst case outcome of this limitation is that the model may combine details from multiple sources producing responses that merge outdated and updated information in a misleading manner. According to the MIT Technology Review, these issues occur because RAG systems may misinterpret the data they retrieve.[2]'),
 Document(id='c2cc5fd3-b69e-4163-bbff-c6cdc44d358a', metadata={'source': 'https://en.wikipedia.org/wiki/Retr

In [31]:
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
context_text

'RAG systems may retrieve factually correct but misleading sources, leading to errors in interpretation. In some cases, an LLM may extract statements from a source without considering its context, resulting in an incorrect conclusion. Additionally, when faced with conflicting information RAG models may struggle to determine which source is accurate. The worst case outcome of this limitation is that the model may combine details from multiple sources producing responses that merge outdated and updated information in a misleading manner. According to the MIT Technology Review, these issues occur because RAG systems may misinterpret the data they retrieve.[2]\n\nLLMs with RAG are programmed to prioritize new information. This technique has been called "prompt stuffing." Without prompt stuffing, the LLM\'s input is generated by a user; with prompt stuffing, additional relevant context is added to this input to guide the model’s response. This approach provides the LLM with key information 

In [32]:
prompt = template.invoke({'context':context_text, 'query':query})
prompt

StringPromptValue(text='\n        you are an knowledgeable assistant!\n        answer to the asked question based on the given context.\n        if not sure, clearly say i don\'t know!\n\n        RAG systems may retrieve factually correct but misleading sources, leading to errors in interpretation. In some cases, an LLM may extract statements from a source without considering its context, resulting in an incorrect conclusion. Additionally, when faced with conflicting information RAG models may struggle to determine which source is accurate. The worst case outcome of this limitation is that the model may combine details from multiple sources producing responses that merge outdated and updated information in a misleading manner. According to the MIT Technology Review, these issues occur because RAG systems may misinterpret the data they retrieve.[2]\n\nLLMs with RAG are programmed to prioritize new information. This technique has been called "prompt stuffing." Without prompt stuffing, th

In [33]:
response = llm_model.invoke(prompt)
response

AIMessage(content='Yes, RAG (retriever) is a part of AI/ML. It is a method of improving large language models (LLMs) by incorporating information retrieval before generating responses.', additional_kwargs={}, response_metadata={'model': 'gemma:2b', 'created_at': '2025-12-20T19:04:21.9472535Z', 'done': True, 'done_reason': 'stop', 'total_duration': 61335532900, 'load_duration': 358824400, 'prompt_eval_count': 551, 'prompt_eval_duration': 51594839200, 'eval_count': 37, 'eval_duration': 9258171500, 'logprobs': None, 'model_name': 'gemma:2b', 'model_provider': 'ollama'}, id='lc_run--019b3d25-1b14-7c51-abb3-ef4936444fb3-0', usage_metadata={'input_tokens': 551, 'output_tokens': 37, 'total_tokens': 588})